<a href="https://colab.research.google.com/github/cagBRT/Data/blob/main/Cost_Sensitive_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cost Sensitive DNNs**

Deep neural networks operate on the assumption that all classes are balanced. <br>
To use DNNs with imbalanced datasets we can use backpropagation to weight exampless from the minority classs.

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where
from sklearn.metrics import roc_auc_score
from keras.layers import Dense
from keras.models import Sequential

**Create an imbalanced dataset**

In [ ]:
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
n_clusters_per_class=2, weights=[0.99], flip_y=0, random_state=4)
# summarize class distribution
counter = Counter(y)
print(counter)

In [ ]:
for label, _ in counter.items():
  row_ix = where(y == label)[0]
  pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()

**Define functions for preparing the data and defining the model**

In [ ]:
# prepare train and test dataset
def prepare_data():
    # split into train and test
  n_train = 5000
  trainX, testX = X[:n_train, :], X[n_train:, :]
  trainy, testy = y[:n_train], y[n_train:]
  return trainX, trainy, testX, testy
# define the neural network model

def define_model(n_input):
# define model
  model = Sequential()
  # define first hidden layer and visible layer
  model.add(Dense(10, input_dim=n_input, activation='relu',kernel_initializer='he_uniform'))
  # define output layer
  model.add(Dense(1, activation='sigmoid'))
  # define loss and optimizer
  model.compile(loss='binary_crossentropy', optimizer='sgd')
  return model

**Split the data into training and test sets**

In [ ]:
trainX, trainy, testX, testy = prepare_data()
# define the model
n_input = trainX.shape[1]

Create and train the model

In [ ]:
model = define_model(n_input)
# fit model
model.fit(trainX, trainy, epochs=100, verbose=0)
# make predictions on the test dataset
yhat = model.predict(testX)
# evaluate the ROC AUC of the predictions
score = roc_auc_score(testy, yhat)
print('ROC AUC: %.3f' % score)

# **Train the model with weighted classes**

The performance of the model improves

In [ ]:
weights = {0:1, 1:100} #small weight is for the majority class
history = model.fit(trainX, trainy, class_weight=weights, epochs=100, verbose=0) # evaluate model
yhat_weight = model.predict(testX)
score_weight = roc_auc_score(testy, yhat_weight)
print('ROC AUC: %.3f' % score_weight)

**Assignment**<br>
Modify the dataset:<br>
- change the size
- change the number of features
-change the class ratio <br>

Note the changes in performance between the two methods